## 第5章: 係り受け解析
日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

まずは指定のデータをダウンロードします。
Google Colaboratoryのセル上で下記のコマンドを実行すると、カレントディレクトリに対象のファイルがダウンロードされます。

In [1]:
%load_ext lab_black

In [2]:
!wget https://nlp100.github.io/data/ai.ja.zip
!unzip ai.ja.zip

--2021-12-11 10:25:59--  https://nlp100.github.io/data/ai.ja.zip
Resolving nlp100.github.io (nlp100.github.io)... 185.199.111.153, 185.199.110.153, 185.199.108.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17516 (17K) [application/zip]
Saving to: ‘ai.ja.zip’

ai.ja.zip           100%[===================>]  17.11K  82.6KB/s    in 0.2s    

2021-12-11 10:25:59 (82.6 KB/s) - ‘ai.ja.zip’ saved [17516/17516]

/usr/bin/sh: 1: unzip: not found


CaboChaおよびCaboChaの実行に必要なMeCabとCRF++をインストールします。

In [3]:
# CRF++のソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7QVR6VXJ5dWExSTQ"
FILE_NAME = "crfpp.tar.gz"
!wget 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O $FILE_NAME
!tar xvf crfpp.tar.gz
%cd CRF++-0.58
!./configure && make && make install && ldconfig
%cd ..

--2021-12-11 10:26:01--  https://docs.google.com/uc?export=download&id=$FILE_ID
Resolving docs.google.com (docs.google.com)... 142.250.76.142, 2404:6800:4004:824::200e
Connecting to docs.google.com (docs.google.com)|142.250.76.142|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-12-11 10:26:02 ERROR 404: Not Found.

tar: This does not look like a tar archive

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now
[Errno 2] No such file or directory: 'CRF++-0.58'
/work/notebooks
/usr/bin/sh: 1: ./configure: not found
/work


In [4]:
# CaboChaのソースファイルのダウンロード・解凍・インストール
FILE_ID = "0B4y35FiV1wh7SDd1Q1dUQkZQaUU"
FILE_NAME = "cabocha-0.69.tar.bz2"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$FILE_ID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$FILE_ID" -O $FILE_NAME && rm -rf /tmp/cookies.txt
!tar -xvf cabocha-0.69.tar.bz2
%cd cabocha-0.69
!./configure -with-charset=utf-8 && make && make check && make install && ldconfig
%cd ..

--2021-12-11 10:26:06--  https://docs.google.com/uc?export=download&confirm=&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU
Resolving docs.google.com (docs.google.com)... 142.250.76.142, 2404:6800:4004:824::200e
Connecting to docs.google.com (docs.google.com)|142.250.76.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘cabocha-0.69.tar.bz2’

cabocha-0.69.tar.bz     [ <=>                ]   3.20K  --.-KB/s    in 0.02s   

2021-12-11 10:26:06 (136 KB/s) - ‘cabocha-0.69.tar.bz2’ saved [3281]

bzip2: (stdin) is not a bzip2 file.
tar: Child returned status 2
tar: Error is not recoverable: exiting now
[Errno 2] No such file or directory: 'cabocha-0.69'
/work
/usr/bin/sh: 1: ./configure: not found
/


係り受け解析を行います。
以下のコマンドを実行することにより、ai.ja.txtを係り受け解析した結果が、ai.ja.txt.parsedとして出力されます。

In [5]:
!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt

/usr/bin/sh: 1: cabocha: not found


出力結果を確認します。

In [6]:
# 行数の確認
!wc -l ./ai.ja.txt.parsed

wc: ./ai.ja.txt.parsed: No such file or directory


In [7]:
# 先頭15行の確認
!head -15 ./ai.ja.txt.parsed

head: cannot open './ai.ja.txt.parsed' for reading: No such file or directory


#### 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [8]:
class Morph:
    def __init__(self, morph):
        surface, attr = morph.split("\t")
        attr = attr.split(",")
        self.surface = surface
        self.base = attr[6]
        self.pos = attr[0]
        self.pos1 = attr[1]

In [9]:
filename = './ai.ja.txt.parsed'

sentences = []
morphs = []
with open(filename, mode='r') as f:
  for line in f:
    if line[0] == '*':  # 係り受け関係を表す行：スキップ
      continue
    elif line != 'EOS\n':  # 文末以外：Morphを適用し形態素リストに追加
      morphs.append(Morph(line))
    else:  # 文末：形態素リストを文リストに追加
      sentences.append(morphs)
      morphs = []

# 確認
for m in sentences[2]:
  print(vars(m))

FileNotFoundError: [Errno 2] No such file or directory: './ai.ja.txt.parsed'

#### 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．

文章は文(sentence)オブジェクトのリストで表され、文オブジェクトは文節(chunk)オブジェクトのリストを要素に持ち、文節オブジェクトは形態素(morph)オブジェクトのリストを要素に持つ階層構造を考え、ここでは指定のクラスChunkに加え、Sentenceを実装しています。
なお、Chunkオブジェクトの要素である係り元文節インデックス番号のリスト(srcs)の作成には1文のすべての文節情報を必要とするため、Sentenceオブジェクトの初期化時に作成しています。

In [ ]:
class Chunk():
  def __init__(self, morphs, dst):
    self.morphs = morphs
    self.dst = dst
    self.srcs = []


class Sentence():
  def __init__(self, chunks):
    self.chunks = chunks
    for i, chunk in enumerate(self.chunks):
      if chunk.dst != -1:
        self.chunks[chunk.dst].srcs.append(i)

In [ ]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']


class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs    # 形態素（Morphオブジェクト）のリスト
        self.dst = dst          # 係り先文節インデックス番号
        self.srcs = []          # 係り元文節インデックス番号のリスト


def parse_cabocha(block):
    def check_create_chunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            res.append(c)
            tmp = []
        return tmp

    res = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = check_create_chunk(tmp)
        elif line[0] == '*':
            dst = line.split(' ')[2].rstrip('D')
            tmp = check_create_chunk(tmp)
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            lineDict = {
                'surface': surface,
                'base': attr[6],
                'pos': attr[0],
                'pos1': attr[1]
            }
            tmp.append(Morph(lineDict))

    for i, r in enumerate(res):
        res[int(r.dst)].srcs.append(i)
    return res

In [ ]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]
for m in blocks[7]:
    print([mo.surface for mo in m.morphs], m.dst, m.srcs)

#### 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [ ]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if int(m.dst) > -1:
            print(''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs]),
                  ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs]), sep='\t')

#### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [ ]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if int(m.dst) > -1:
            pre_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
            pre_pos = [mo.pos for mo in m.morphs]
            post_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs])
            post_pos = [mo.pos for mo in b[int(m.dst)].morphs]
            if '名詞' in pre_pos and '動詞' in post_pos:
                print(pre_text, post_text, sep='\t')

#### 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

係り元と係り先の文節のペアを作成し、pydotのgraph_from_edgesに渡すことで、グラフを作成しています。
なお、表層形そのままでは1文内に同じ文字列の文節が複数回出てきた場合に区別できないため、末尾にIDを付与して表示しています。



In [ ]:
filename = './ai.ja.txt.parsed'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

pairs = []
target = blocks[7]
for m in target:
    if int(m.dst) > -1:
        pre_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
        post_text = ''.join([mo.surface if mo.pos != '記号' else '' for mo in target[int(m.dst)].morphs])
        pairs.append([pre_text, post_text])

print(pairs)
g = pydot.graph_from_edges(pairs)
g.write_png('./ans44.png', prog='dot')

#### 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

* 動詞を含む文節において，最左の動詞の基本形を述語とする
* 述語に係る助詞を格とする
* 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．
* コーパス中で頻出する述語と格パターンの組み合わせ
* 「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [ ]:
with open('./ans45.txt', 'w') as f:
  for sentence in sentences:
    for chunk in sentence.chunks:
      for morph in chunk.morphs:
        if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
          cases = []
          for src in chunk.srcs:  # 見つけた動詞の係り元chunkから助詞を探す
            cases = cases + [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == '助詞']
          if len(cases) > 0:  # 助詞が見つかった場合は重複除去後辞書順にソートして出力
            cases = sorted(list(set(cases)))
            line = '{}\t{}'.format(morph.base, ' '.join(cases))
            print(line, file=f)
          break

In [ ]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']


class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs    # 形態素（Morphオブジェクト）のリスト
        self.dst = dst          # 係り先文節インデックス番号
        self.srcs = []          # 係り元文節インデックス番号のリスト


def parse_cabocha(block):
    def check_create_chunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            res.append(c)
            tmp = []
        return tmp

    res = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = check_create_chunk(tmp)
        elif line[0] == '*':
            dst = line.split(' ')[2].rstrip('D')
            tmp = check_create_chunk(tmp)
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            lineDict = {
                'surface': surface,
                'base': attr[6],
                'pos': attr[0],
                'pos1': attr[1]
            }
            tmp.append(Morph(lineDict))

    for i, r in enumerate(res):
        res[int(r.dst)].srcs.append(i)
    return res

In [ ]:
filename = './ans45.txt'
with open(filename, mode='r', encoding='utf-8') as f:
    blocks = f.read().split('EOS\n')
blocks = list(filter(lambda x: x != '', blocks))
blocks = [parse_cabocha(block) for block in blocks]

for b in blocks:
    for m in b:
        if len(m.srcs) > 0:
            pre_morphs = [b[int(s)].morphs for s in m.srcs]
            pre_morphs = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in pre_morphs]
            pre_surface = [[p.surface for p in pm] for pm in pre_morphs]
            pre_surface = list(filter(lambda x: x != [], pre_surface))
            pre_surface = [p[0] for p in pre_surface]
            post_base = [mo.base for mo in m.morphs]
            post_pos = [mo.pos for mo in m.morphs]
            if len(pre_surface) > 0 and '動詞' in post_pos:
                print(post_base[0], ' '.join(pre_surface), sep='\t')

In [ ]:
# 確認
!cat ./ans45.txt | sort | uniq -c | sort -nr | head -n 10

In [ ]:
!cat ./ans45.txt | grep '行う' | sort | uniq -c | sort -nr | head -n 5

In [ ]:
!cat ./ans45.txt | grep 'なる' | sort | uniq -c | sort -nr | head -n 5

In [ ]:
!cat ./ans45.txt | grep '与える' | sort | uniq -c | sort -nr | head -n 5

#### 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

* 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
* 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

In [ ]:
with open('./ans46.txt', 'w') as f:
  for sentence in sentences:
    for chunk in sentence.chunks:
      for morph in chunk.morphs:
        if morph.pos == '動詞':  # chunkの左から順番に動詞を探す
          cases = []
          modi_chunks = []
          for src in chunk.srcs:  # 見つけた動詞の係り元chunkから助詞を探す
            case = [morph.surface for morph in sentence.chunks[src].morphs if morph.pos == '助詞']
            if len(case) > 0:  # 助詞を含むchunkの場合は助詞と項を取得
              cases = cases + case
              modi_chunks.append(''.join(morph.surface for morph in sentence.chunks[src].morphs if morph.pos != '記号'))
          if len(cases) > 0:  # 助詞が1つ以上見つかった場合は重複除去後辞書順にソートし、項と合わせて出力
            cases = sorted(list(set(cases)))
            line = '{}\t{}\t{}'.format(morph.base, ' '.join(cases), ' '.join(modi_chunks))
            print(line, file=f)
          break

In [ ]:
# 確認
!cat ./ans46.txt | head -n 10